# Build Excel files

Refer to the [Tally documentation on building Excel files](https://python.datasmoothie.com/7_build_excel_output.html) for tips on how to control the layout and presentation of your build.

<style>
.alert{
    padding:10px;
    margin-bottom:10px;
    background-color: rgb(207, 226, 255);
    border: 1px solid rgb(158,197,254);
    border-radius: 5px;
    width:97%;
    margin-top:20px;
}
</style>

<div class="alert"><p>
By default, every table goes into a seperate sheet. If you want all tables in one sheet, set <code>one_sheet_per_table</code> to <code>False</code>.
</p></div>

In [16]:
import tally
import os
import pandas as pd
import json
ds = tally.DataSet(api_key=os.environ.get("TALLY_KEY"))

In [ ]:
project_name = "tally_example_project"
one_sheet_per_table = True
banner_variables = []
stub_variables = []

In [17]:
ds.use_quantipy(f"../data/4_{project_name}.json",f"../data/4_{project_name}.csv")
variables = ds.variables()


# Build starts here

## 1. Define your banner variables (to appear at the top of your file)

#### this only runs if you haven't specified banner variables yourself - we get some common variable names

In [19]:
if len(banner_variables) == 0:
  with open('../specs/common_variable_names.json', 'r') as f:
      common_variables = json.load(f)
  common_variables = [i.lower() for i in common_variables]
  for var in variables['single']:
    if any(match in var for match in common_variables):
      banner_variables.append(var)

# 2. Define stub/side variables

These will apear on the left.

In [20]:
# you can define your stub_variables in here or in pipeline.ipynb
# stub_variables = ['q1',
#  'q2',
#  'q3',
#  'q4',
#  'q5'
#]

#### if you haven't defined the list, we add all questions except the banner questions

In [ ]:
if len(stub_variables) == 0:
  stub_variables = variables['single']
  stub_variables = [i for i in stub_variables if i not in banner_variables]

# 3. Build the Excel table

In [22]:
build = tally.Build(name='client A', default_dataset=ds)

####
# Set options sthat apply to whole build/Excel file
####
# set weight
#build.options.set_weight('weight')
# set base outside of main table
build.options.set_base_position('outside')
# brand with font
build.options.set_font('open sans')
# only show percentage results
build.options.set_ci(['c%'])

In [ ]:
if one_sheet_per_table:
    build.options.set_question_format('percentage', {"bold":True})
    build.options.set_column_format_for_type('base', 1, {"bold":True})
    build.options.set_column_format_for_type('percentage', 1, {"bold":True})

    for questionlist_index, question in enumerate(stub_variables):
        sheet = build.add_sheet(name=question, banner=banner_variables)
        if questionlist_index ==0:
            sheet.add_table(stub={'x' : question, 'xtotal':True, 'decimals':0})
        else:
            sheet.options.set_base_position('hide')
            sheet.add_table(stub={'x' : question, 'xtotal':True, 'decimals':0})

else:

    sheet = build.add_sheet(banner=banner_variables)

    ####
    # Set options that apply on a sheet-level
    ####

    # set color and weight of base
    sheet.options.set_answer_format(
        'base', 
        {"font_color":"F15A30", "bold":True,'text_wrap': True}
    )
    sheet.options.set_question_format('percentage', {"bold":True})
    sheet.options.set_column_format_for_type('base', 1, {"bold":True})
    sheet.options.set_column_format_for_type('percentage', 1, {"bold":True})

    ####
    # Run the calculations and add them to a build
    ####
    for questionlist_index, question in enumerate(stub_variables):
        if questionlist_index ==0:
            sheet.add_table(stub={'x' : question, 'xtotal':True, 'decimals':0})
        else:
            sheet.options.set_base_position('hide')
            sheet.add_table(stub={'x' : question, 'xtotal':True, 'decimals':0})

In [23]:
build.save_excel(f'../outputs/{project_name}.xlsx')

In [24]:
ds.write_quantipy(f"../data/5_{project_name}.json",f"../data/5_{project_name}.csv")